In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_path = "/content/drive/MyDrive/Colab Notebooks/Deep Learning/Sentiment_classification/"
%cd /content/drive/MyDrive/Colab Notebooks/Deep Learning/Sentiment_classification/

/content/drive/MyDrive/Colab Notebooks/Deep Learning/Sentiment_classification


# Install dependencies

In [ ]:
!pip install transformers
!pip install vncorenlp
!pip install fairseq
!pip install fastBPE
!pip install sentencepiece


     |████████████████████████████████| 3.8 MB 12.7 MB/s 
     |████████████████████████████████| 895 kB 44.1 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 41.1 MB/s 
     |████████████████████████████████| 6.5 MB 42.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 2.6 MB 13.3 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=c2ae9ba2c876c95eba94118d30c82285a0c74c876fc0f2b8dec36441a416ddd7
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
     |████████████████████████████████| 1.7 MB 12.2 MB/s 
     |████████████████████████████████| 145 kB 46.0 MB/s 
     |████████████████████████████████| 90 kB 8.4 MB/s 
     |████████████████████████████████|

  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483193 sha256=40ba9d263b198b415129c3f67c8a634a0e01405e27ede12feced2069e72f51e4
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
Successfully built fastBPE
     |████████████████████████████████| 1.2 MB 11.8 MB/s 


In [ ]:
!pip install clean-text[gpl,sklearn]

     |████████████████████████████████| 53 kB 1.1 MB/s 
     |████████████████████████████████| 175 kB 23.0 MB/s 
     |████████████████████████████████| 235 kB 41.4 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=fac10b7c23b86f64426b2c396106073363cc71af96d572aa72da5ed574c170ff
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


# Install Pre-trained model

In [ ]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz


--2022-03-18 08:14:41--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 52.84.121.103, 52.84.121.122, 52.84.121.44, ...
Connecting to public.vinai.io (public.vinai.io)|52.84.121.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘PhoBERT_base_transformers.tar.gz’

PhoBERT_base_transf 100%[===================>] 307.47M  31.4MB/s    in 9.4s    

2022-03-18 08:14:51 (32.6 MB/s) - ‘PhoBERT_base_transformers.tar.gz’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [ ]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/


VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  37.0MB/s    in 0.7s    
vi-vocab            100%[===================>] 514.20K  --.-KB/s    in 0.04s   
wordsegmenter.rdr   100%[===================>] 125.50K  --.-KB/s    in 0.02s   


# Import library

In [ ]:
import os  
import argparse
import warnings

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import TensorDataset, DataLoader
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from vncorenlp import VnCoreNLP
from transformers import RobertaConfig, BertPreTrainedModel, RobertaModel, AdamW
from transformers import get_linear_schedule_with_warmup, get_constant_schedule
from transformers.modeling_utils import *
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from tqdm import tqdm_notebook
from tqdm import tqdm

tqdm.pandas()

warnings.filterwarnings("ignore")


In [ ]:
train_path = os.path.join(base_path, "data/train.csv")
dict_path = os.path.join(base_path, "PhoBERT_base_transformers/dict.txt")
config_path = os.path.join(base_path, "PhoBERT_base_transformers/config.json")
# pretrained_path = os.path.join(base_path, "PhoBERT_base_transformers/model.bin")
pretrained_path = os.path.join(base_path, "models/model_7.bin")
bpe_codes = os.path.join(base_path, "PhoBERT_base_transformers/bpe.codes")
checkpoint_path = os.path.join(base_path, "models/")
rdrsegmenter_path = os.path.join(base_path, "vncorenlp/VnCoreNLP-1.1.1.jar")


# Utils

In [ ]:
def convert_lines(df, vocab, bpe, max_sequence_length):
    # Initial output matrix
    outputs = np.zeros((len(df), max_sequence_length))
    
    cls_id = 0
    pad_id = 1
    eos_id = 2

    for (idx, text, label, _) in tqdm(df.itertuples(name=None), total=len(df)):
        subwords = bpe.encode('<s> '+ text +' </s>')
        input_ids = vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
        if len(input_ids) > max_sequence_length: 
            input_ids = input_ids[:max_sequence_length] 
            input_ids[-1] = eos_id
        else:
            input_ids = input_ids + [pad_id, ] * (max_sequence_length - len(input_ids))

        outputs[idx, :] = np.array(input_ids)
    return outputs

def seed_everything(SEED):
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

def sigmoid(x):
    return 1 / (1 + np.exp(-x))
    

In [ ]:
import pickle

def _save_pkl(path, obj):
  with open(path, 'wb') as f:
    pickle.dump(obj, f)

def _load_pkl(path):
  with open(path, 'rb') as f:
    obj = pickle.load(f)
  return obj


# Preprocessing

In [ ]:
# Khởi tạo Byte Pair Encoding cho PhoBERT
class BPE():
  bpe_codes = bpe_codes

bpe = fastBPE(BPE())
rdrsegmenter = VnCoreNLP(rdrsegmenter_path, annotators="wseg", max_heap_size='-Xmx500m')


In [ ]:
# Load the Dictionary
vocab = Dictionary()
vocab.add_from_file(dict_path)


In [ ]:
text = "Ông Nguyễn Khắc Chúc đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

# To perform word (and sentence) segmentation
' '.join([' '.join(sent) for sent in rdrsegmenter.tokenize(text)])
 

'Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội . Bà Lan , vợ ông Chúc , cũng làm_việc tại đây .'

In [ ]:
# Load data

train_df = pd.read_csv(train_path)
print("Num samples: {}".format(len(train_df)))



Num samples: 23452


In [ ]:
# Count the number of words
train_df['number_of_words'] = train_df['text'].progress_apply(lambda x: len(str(x).strip().split()))
train_df.head()


100%|██████████| 23452/23452 [00:00<00:00, 59264.16it/s]


,text,label,number_of_words
0,Xuất hiện trên thị trường khi vừa bước sang gi...,Positive,51
1,Khép lại năm 2014. Nhìn lại chặn đường đã đi q...,Positive,64
2,Bên cạnh Phú Quốc (Kiên Giang) và Bắc Vân Phon...,Positive,43
3,"Vi vu Nha Trang, nghỉ dưỡng tại Biệt thự Vinpe...",Positive,54
4,"Sở hữu nhiều giá trị vượt trội về diện tích, k...",Positive,49


In [ ]:
train_df["number_of_words"].describe()

count    23452.000000
mean       159.640713
std        300.892126
min          1.000000
25%         32.000000
50%         64.000000
75%        149.000000
max       4324.000000
Name: number_of_words, dtype: float64

In [ ]:
# number of rows with text length < 2
no_text = train_df[train_df['number_of_words'] < 2]
print("Number of rows with no text {}".format(len(no_text)))
# drop these rows
train_df.drop(no_text.index, inplace=True)

# number of rows that text is NaN
null_text = train_df[train_df['text'].isnull()]
print("Number of rows with null text {}".format(len(null_text)))
# drop these rows
train_df.drop(null_text.index, inplace=True)

# number of rows with text length < 2
much_text = train_df[train_df['number_of_words'] > 500]
print("Number of rows with text more than 500 words {}".format(len(much_text)))
# drop these rows
train_df.drop(much_text.index, inplace=True)


Number of rows with no text 8
Number of rows with null text 0
Number of rows with text more than 500 words 1581


In [ ]:
import re
import string
from cleantext import clean


def clean_text(text):
    text = clean(text,
      fix_unicode=False,               # fix various unicode errors
      to_ascii=False,                  # transliterate to closest ASCII representation
      lower=False,                     # lowercase text
      no_emoji=True,
      no_line_breaks=True,             # fully strip line breaks as opposed to only normalizing them
      no_urls=True,                    # replace all URLs with a special token
      no_emails=True,                  # replace all email addresses with a special token
      no_phone_numbers=True,           # replace all phone numbers with a special token
      no_numbers=False,                # replace all numbers with a special token
      no_digits=False,                 # replace all digits with a special token
      no_currency_symbols=True,        # replace all currency symbols with a special token
      no_punct=False,                  # remove punctuations
      replace_with_url=" ",
      replace_with_email=" ",
      replace_with_currency_symbol=" ",
      lang="vi"                        # set to 'de' for German special handling
  )
    return text.strip()

In [ ]:
train_df["text"] = train_df['text'].progress_apply(lambda x: clean_text(x))

100%|██████████| 21863/21863 [00:26<00:00, 815.53it/s]


In [ ]:
# number of rows with text length < 2
no_text = train_df[train_df['number_of_words'] < 2]
print("Number of rows with no text {}".format(len(no_text)))
# drop these rows
train_df.drop(no_text.index, inplace=True)

# number of rows that text is NaN
null_text = train_df[train_df['text'].isnull()]
print("Number of rows with null text {}".format(len(null_text)))
# drop these rows
train_df.drop(null_text.index, inplace=True)

Number of rows with no text 0
Number of rows with null text 0


In [ ]:
null_label = train_df[train_df['label'].isnull()]
print("Number of rows with null label {}".format(len(null_label)))
# drop these rows
train_df.drop(null_label.index, inplace=True)

Number of rows with null label 0


In [ ]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [ ]:
train_df

,text,label,number_of_words
0,Cơ sở trên đã vi phạm và tái phạm nhiều lần bị...,Negative,58
1,Tiến độ dự án Opal Garden tháng 07,Positive,8
2,Vậy là capitaland chuẩn bị triển khai dự án mớ...,Positive,38
3,Dự kiến tháng 4 khởi công dự án đường ven sông...,Positive,202
4,Tắm trắng cực nhanh bằng bia không lo bắt nắng,Positive,10
...,...,...,...
21858,"Mới đây, Thủ tướng Chính phủ đã đồng ý về chủ ...",Negative,34
21859,"Không chỉ nổi trội với các lợi thế về vị trí, ...",Positive,63
21860,Sự kiện mở bán đất nền dự án Mandarin Residenc...,Positive,18
21861,Mẫu nhà có gác xép đơn giản Cre: Groups: Nhà đ...,Negative,17


In [ ]:
# Tokenize
train_df["text"] = train_df["text"].progress_apply(lambda x: ' '.join([' '.join(sent) for sent in rdrsegmenter.tokenize(x)]))


100%|██████████| 21863/21863 [02:08<00:00, 169.88it/s]


In [ ]:
train_df["text"] = train_df["text"].progress_apply(lambda x: x.lower())

100%|██████████| 21863/21863 [00:00<00:00, 204536.08it/s]


In [ ]:
train_df.to_csv(os.path.join(base_path, "data/train_segment.csv"), index=False, encoding="utf-8")

In [ ]:
train_df = pd.read_csv(os.path.join(base_path, "data/train_segment.csv"))

In [ ]:
max_sequence_length = 256

X_train = convert_lines(train_df, vocab, bpe, max_sequence_length)
y = train_df["label"].values
lb = LabelEncoder()
y = lb.fit_transform(y)


100%|██████████| 21863/21863 [00:18<00:00, 1187.70it/s]


In [ ]:
_save_pkl(os.path.join(base_path, "data/X_train_raw.pkl"), X_train)
_save_pkl(os.path.join(base_path, "data/y.pkl"), y)


# Training

In [ ]:
X_train = _load_pkl(os.path.join(base_path, "data/X_train_raw.pkl"))
y = _load_pkl(os.path.join(base_path, "data/y.pkl"))


In [ ]:
seed = 69
seed_everything(seed)

if torch.cuda.is_available():
    print('Using GPU')
    device = torch.device('cuda')
else:
    print('Using CPU')
    device = torch.device('cpu')


Using GPU


In [ ]:
# Define the model
class RobertaVN(BertPreTrainedModel):
   config_class = RobertaConfig
   base_model_prefix = "roberta"
   def __init__(self, config):
       super(RobertaVN, self).__init__(config)
       self.num_labels = config.num_labels
       self.roberta = RobertaModel(config)
       self.classifier = nn.Linear(4 * config.hidden_size, self.num_labels)
      #  self.classifier = nn.Linear(config.hidden_size, self.num_labels)

       self.init_weights()

   def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):

       outputs = self.roberta(input_ids,
                            attention_mask=attention_mask,
#                           token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask)
       cls_output = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]), dim=-1)
      #  cls_output = outputs[2][-1][:,0, ...]
       logits = self.classifier(cls_output)
       return logits


In [ ]:
# Load model
config = RobertaConfig.from_pretrained(
    config_path,
    output_hidden_states=True,
    num_labels=1
)

model = RobertaVN.from_pretrained(pretrained_path, config=config)
# model.eval()
model.to(device)


You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/drive/MyDrive/Colab Notebooks/Deep Learning/Sentiment_classification/models/model_7.bin were not used when initializing RobertaVN: ['module.roberta.encoder.layer.7.attention.self.query.weight', 'module.roberta.encoder.layer.2.intermediate.dense.bias', 'module.roberta.encoder.layer.3.attention.output.dense.bias', 'module.roberta.embeddings.position_embeddings.weight', 'module.roberta.encoder.layer.3.attention.output.dense.weight', 'module.roberta.encoder.layer.6.attention.self.value.bias', 'module.roberta.encoder.layer.4.attention.self.key.bias', 'module.roberta.encoder.layer.11.output.dense.weight', 'module.roberta.encoder.layer.8.attention.self.query.weight', 'module.roberta.encoder.layer.10.attention.output.dense.bias', 'module.roberta.encoder.layer.2.attention.output.dense.bias', 'modu

RobertaVN(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=0)
      (position_embeddings): Embedding(258, 768, padding_idx=0)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,),

In [ ]:
if torch.cuda.device_count():
    print(f"Training using {torch.cuda.device_count()} gpus")
    model = nn.DataParallel(model)
    tsfm = model.module.roberta
else:
    tsfm = model.roberta


Training using 1 gpus


In [ ]:
batch_size = 24
epochs = 20
lr = 1e-5
accumulation_steps = 5

# Optimizer and lr schedulers
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
num_train_optimization_steps = int(epochs * len(X_train) / batch_size / accumulation_steps)
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, correct_bias=False)
# Warmup 100 step before start learning
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_train_optimization_steps)
scheduler0 = get_constant_schedule(optimizer)


In [ ]:
# Checkpoint path 
if not os.path.exists(checkpoint_path):
    os.mkdir(checkpoint_path)
    

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, stratify=y)


In [ ]:
y_train.shape

(17490,)

In [ ]:

best_score = 0
num_early = 0
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.long), torch.tensor(y_train, dtype=torch.long))
valid_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.long), torch.tensor(y_val, dtype=torch.long))
for child in tsfm.children():
    for param in child.parameters():
        if not param.requires_grad:
            pass
        param.requires_grad = False

frozen = True
for epoch in range(epochs):
    if epoch > 0 and frozen:
        for child in tsfm.children():
            for param in child.parameters():
                param.requires_grad = True
        frozen = False
        del scheduler0
        torch.cuda.empty_cache()
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    total_loss = 0.
    train_preds = None
    
    optimizer.zero_grad()
    model.train()   
    with tqdm_notebook(enumerate(train_loader), total=len(train_loader), unit="batch") as process_bar:
        for step, (x_batch, y_batch) in process_bar:
            process_bar.set_description(f"Epoch {epoch}...")
            
            y_pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device))
            predictions = y_pred.squeeze().detach().cpu().numpy()
            train_preds = np.atleast_1d(predictions) if train_preds is None else np.concatenate([train_preds, np.atleast_1d(predictions)])
            # compute loss
            loss = F.binary_cross_entropy_with_logits(y_pred.view(-1).to(device), y_batch.float().to(device))
            loss = loss.mean()
            loss.backward()

            if step % accumulation_steps == 0 or step == len(train_loader) - 1:
                optimizer.step()
                optimizer.zero_grad()
                if not frozen:
                    scheduler.step()
                else:
                    scheduler0.step()
            process_bar.set_postfix(loss=loss.item())
            total_loss += loss.item()
    train_preds = sigmoid(train_preds)
    train_acc = accuracy_score(y_train, train_preds > 0.5) * 100
    train_f1 = f1_score(y_train, train_preds > 0.5)
    print("Accuracy: {0:.4f}%".format(train_acc))
    print("F1 Score: {0:.4f}".format(train_f1))
    print("Average training loss: {0:.4f}".format(total_loss))

    model.eval()
    val_preds = None
    process_bar = tqdm(enumerate(valid_loader), total=len(valid_loader), desc="Validating...")
    for step, (x_batch, y_batch) in process_bar:
        with torch.no_grad():
            y_pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device))
            y_pred = y_pred.squeeze().detach().cpu().numpy()
            # convert to 1d array
            val_preds = np.atleast_1d(y_pred) if val_preds is None else np.concatenate([val_preds, np.atleast_1d(y_pred)])
    val_preds = sigmoid(val_preds)

    score = f1_score(y_val, val_preds > 0.5)
    val_accuracy = accuracy_score(y_val, val_preds > 0.5)
    print(f"\nAUC: {roc_auc_score(y_val, val_preds):.4f}, F1 score @0.5: {score:.4f}, Accuracy: {round(val_accuracy * 100, 2)}%")
    if score >= best_score:
        torch.save(model.state_dict(), os.path.join(checkpoint_path, f"model_{epoch}.bin"))
        best_score = score
    else:
        num_early += 1
    
    if num_early > 10:
        print("Early stopping at epoch {}".format(epoch))
        break


  0%|          | 0/729 [00:00<?, ?batch/s]

Accuracy: 57.8559%
F1 Score: 0.7116
Average training loss: 509.8706


Validating...: 100%|██████████| 183/183 [02:18<00:00,  1.32it/s]



AUC: 0.5136, F1 score @0.5: 0.7645, Accuracy: 61.88%


  0%|          | 0/729 [00:00<?, ?batch/s]

...

Accuracy: 59.4854%
F1 Score: 0.7335
Average training loss: 475.3132


Validating...: 100%|██████████| 183/183 [02:18<00:00,  1.32it/s]



AUC: 0.7075, F1 score @0.5: 0.7673, Accuracy: 62.89%


  0%|          | 0/729 [00:00<?, ?batch/s]

KeyboardInterrupt: ignored